In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
model_name = "E:\\models\\Llama-3.2-3B-Instruct"
token = os.getenv("HFTOKEN")
import torch

In [2]:
from transformers import AutoModelForCausalLM 
from transformers import AutoTokenizer
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

c:\Users\khatt\Documents\AmazonEcho-NLP-Project\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:13<00:00,  6.73s/it]


In [3]:
from transformers import pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    return_full_text=True
)

Device set to use cpu


In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings
model_name = "E:/models/gte-small"
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    multi_process=False,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
    
)


C:\Users\khatt\AppData\Local\Temp\ipykernel_26268\2838025220.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In [6]:
import sys
import os

# Go up one level (or more depending on where your .ipynb file is)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))  # adjust if needed
sys.path.append(project_root)
from WebsiteScrap import WebsiteScrapper
web = WebsiteScrapper("https://www.amazon.com/Amazon-Basics-Everyday-Plates-Disposable/dp/B0C2CY22B8/?_encoding=UTF8&pd_rd_w=APjaP&content-id=amzn1.sym.f2128ffe-3407-4a64-95b5-696504f68ca1&pf_rd_p=f2128ffe-3407-4a64-95b5-696504f68ca1&pf_rd_r=YXMEXN435CNCQZWD512A&pd_rd_wg=7Izqy&pd_rd_r=3fbb0d02-8f22-4b11-ac45-f11f9ef282d6&ref_=pd_hp_d_btf_crs_zg_bs_284507&th=1")
dfname = web.websitename()


In [7]:
from pinecone import Pinecone
import os
import sys
from dotenv import load_dotenv
envpath = "../.env"
load_dotenv(dotenv_path=envpath)
PINECONE = os.getenv("PINECONE")
pc = Pinecone(api_key=PINECONE)
index = pc.Index("vectorized")
user_input = input("User: ")




In [8]:
user_input = "What do people thinki about the product"
vectorized_input = embedding_model.embed_query(user_input)

context = index.query(
    namespace="echo",
    vector=vectorized_input,
    top_k=2,
    include_metadata=True
)

c:\Users\khatt\Documents\AmazonEcho-NLP-Project\venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [9]:
len(context['matches'])

2

In [32]:
text = [context['matches'][i]['metadata']['text'] for i in range(len(context['matches']))]
combinedtext = str(".".join(text))

In [11]:
prompt = """
You are an assistant that is answer people questions regarding there userinput.
You are also provided context from a source that relates to the userinput
Please think and answer based on the userinput and the context that you are given.
Also provide the answer in a json format
{"userinput":{},
 "context" : 

{context}
"""

In [33]:
combinedtext

'Awesome product! However, it does take time for Alexa to recognize other language when asked to play Spanish music and after a while she’ll turn off by it self..I really enjoy itI'

In [ ]:


messages = [
    {
        "role": "system",
        "content": (
            "You are an Amazon AI Assistant that answers customer questions using reviews from a vector database. "
            "Answer the question using only the reviews provided below, and limit your response to 100 words.\n\n"
            f"Reviews: {combinedtext}"
        )
    },
    {
        "role": "user",
        "content": "What do people think about the product"
    }
]

In [29]:
messages.format(context=combinedtext)

AttributeError: 'list' object has no attribute 'format'

In [38]:
output = pipe(messages)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [46]:
output

[{'generated_text': [{'role': 'system',
    'content': 'You are an Amazon AI Assistant that answers customer questions using context from a vector database. Answer the question using only the context provided below, and limit your response to 100 words.\n\nContext: Awesome product! However, it does take time for Alexa to recognize other language when asked to play Spanish music and after a while she’ll turn off by it self..I really enjoy itI'},
   {'role': 'user', 'content': 'What do people think about the product'},
   {'role': 'assistant',
    'content': 'Based on the context, it seems that the product (likely an Amazon Echo speaker) is generally well-liked. The user calls it an "Awesome product" and mentions they "really enjoy it". However, there is a concern about the product\'s performance, specifically its ability to recognize and play music in multiple languages, including Spanish. Some users may experience issues with the device turning off after a while, but this doesn\'t seem

In [45]:
print(output[0]['generated_text'][2]['content'])

Based on the context, it seems that the product (likely an Amazon Echo speaker) is generally well-liked. The user calls it an "Awesome product" and mentions they "really enjoy it". However, there is a concern about the product's performance, specifically its ability to recognize and play music in multiple languages, including Spanish. Some users may experience issues with the device turning off after a while, but this doesn't seem to detract from the overall positive sentiment about the product.


In [62]:
print(answer[0]['generated_text'])


You are an assistant that is answer people questions regarding there userinput.
What do people thinki about the product:
You are also provided context from a source that relates to the userinput
Awesome product! However, it does take time for Alexa to recognize other language when asked to play Spanish music and after a while she’ll turn off by it self..I really enjoy itI
Please think and answer based on the userinput and the context that you are given.
You will be asked to provide a name for the product.
I need to know what do you think about something.
I have already answered this question.
I need to know what do you think about something.
I have already answered this question.
I want to know what do you think about something.
I have already answered this question.
I want to know what do you think about something.
I have already answered this question.
I want to know what do you think about something.
I have already answered this question.
I want to know what do you think about some

In [63]:
from WebsiteScrap import WebsiteScrapper
from embeddedstuff import Embedded
from transformers import AutoModelForCausalLM 
from transformers import AutoTokenizer
from transformers import pipeline

class llmmodel:
    def __init__(self,user_input,url):
        self.model_name="E:\\models\\pleasRAG"
        self.user_input = user_input
        self.url = url

    def llmmodel(self):
        model = AutoModelForCausalLM.from_pretrained(self.model_name)
        tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            return_full_text=True
        )

        return pipe
        

    def context(self):
        web = WebsiteScrapper
        emb = Embedded
        embedding_model=emb.load_embmodel()
        dfname = web.websitename
        PINECONE = os.getenv("PINECONE")
        pc = Pinecone(api_key=PINECONE)
        index = pc.Index("amazon")
        
        vectorized_input = embedding_model.embed_query(self.user_input)
        context = index.query(
            namespace=dfname,
            vector=vectorized_input,
            top_k=2,
            include_metadata=True
        )
        combinedcontext = [context['matches'][i]['metadata']['text'] for i in range(len(context['matches']))]
        combinedcontext=str(".".join(combinedcontext))
        return combinedcontext

    def feedback(self):
        contextfromdb = self.context()
        prompt = """What do yu think about the product based on the reviews 
        {reviews}"""
        answer = pipe(prompt.format(review = contextfromdb))

        return answer[0]['generatedtext']

        


